In [21]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [22]:
import warnings
warnings.filterwarnings("ignore")

# 1. Data Preparation

In [23]:
## input FNA set ## 88701 x 20 df
df_ROI_FNA = pd.read_csv(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_NonInFNA\postPredict_tPRfNRfDR_noningestedFNA.csv").drop(columns=["Column1"]).reset_index().drop(columns=["index"])

In [24]:
df_ROI_FNA

,pixel_id,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,609.5114,682.5922,846.5858,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,0.064937,-0.160987,0.157940,0.0,0.0,-0.164278,-0.238513,-0.067201,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.071189,0.458388,0.0,0.0,-0.164278,-0.238513,-0.067201,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.103618,0.411193,0.0,0.0,-0.164278,-0.238513,-0.067201,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,0.192912,0.757167,-0.103978,0.0,0.0,-0.164278,-0.238513,0.932799,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,0.0,-0.265543,0.354649,0.440587,0.0,0.0,-0.164278,-0.238513,-0.067201,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85318,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.084344,0.215631,0.0,0.0,-0.164278,-0.238513,-0.067201,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000
85319,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.470346,0.211699,0.0,0.0,-0.164278,-0.238513,-0.067201,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000
85320,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.420349,-0.103978,0.0,0.0,-0.164278,-0.238513,-0.067201,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000
85321,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,0.0,-0.265543,0.499509,-0.103978,0.0,0.0,-0.164278,-0.238513,-0.067201,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000


In [25]:
df_ROI_FNA["Sample"] = 0
df_ROI_FNA["Row"] = 0
df_ROI_FNA["Scan"] = 0
df_ROI_FNA_Sample = []
df_ROI_FNA_Row = []
df_ROI_FNA_Scan = []
for i in range(len(df_ROI_FNA["pixel_id"])):
    df_ROI_FNA_Sample.append("_".join(str(df_ROI_FNA["pixel_id"][i]).split("_")[:-2]))
    df_ROI_FNA_Row.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-1]))
    df_ROI_FNA_Scan.append(int(str(df_ROI_FNA["pixel_id"][i]).split("_")[-2]))
df_ROI_FNA["Sample"] = df_ROI_FNA_Sample
df_ROI_FNA["Row"] = df_ROI_FNA_Row
df_ROI_FNA["Scan"] = df_ROI_FNA_Scan
pop_column = df_ROI_FNA.pop('Scan')
df_ROI_FNA.insert(1, 'Scan', pop_column)
pop_column = df_ROI_FNA.pop('Row')
df_ROI_FNA.insert(1, 'Row', pop_column)
pop_column = df_ROI_FNA.pop('Sample')
df_ROI_FNA.insert(1, 'Sample', pop_column)

In [26]:
df_ROI_FNA = df_ROI_FNA.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [27]:
df_ROI_FNA

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.0,0.064937,-0.160987,0.157940,0.0,0.0,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,72,132,0.0,0.183426,0.172694,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,72,131,0.0,0.159255,0.170017,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,72,130,0.0,0.159809,0.048192,0.188911,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,72,127,0.0,-0.265543,0.117918,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85318,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5,52,137,0.0,0.068952,-0.160987,-0.103978,0.0,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000
85319,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5,52,138,0.0,-0.265543,0.061230,-0.103978,0.0,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000
85320,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5,52,139,0.0,-0.265543,0.064149,-0.103978,0.0,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000
85321,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5,52,124,0.0,-0.265543,-0.160987,-0.103978,0.0,0.0,...,1,0,1.0,0.0,FN,1.000000,0.000000,0.522016,1.000000,0.000000


In [28]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI = df_ROI_FNA.loc[df_ROI_FNA["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [29]:
sample_ROI_list = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                   HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI, 
                   HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI]

In [30]:
name_list = ["HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI", 
             "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI", 
             "HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI"]

In [31]:
sample_ROI_list_FNA = [HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI, 
                       HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI, 
                       HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI]

In [32]:
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,4,0.0,0.064937,-0.160987,0.157940,0.0,0.0,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
1,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,5,0.0,-0.265543,0.071189,0.458388,0.0,0.0,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
2,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,6,0.0,-0.265543,0.103618,0.411193,0.0,0.0,...,0,1,0.0,1.0,FP,0.406174,0.593826,0.502985,0.376381,0.623619
3,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,194,0.0,-0.265543,0.665530,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
4,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,1,195,0.0,-0.265543,0.605478,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14820,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,144,0.0,0.106642,0.116234,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
14821,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,145,0.0,-0.265543,0.112922,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
14822,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,117,146,0.0,-0.265543,0.099805,-0.103978,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000
14823,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide...,HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3,118,177,0.0,-0.265543,0.484849,0.237960,0.0,0.0,...,0,0,1.0,0.0,TN,1.000000,0.000000,0.522016,1.000000,0.000000


In [33]:
HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI

,pixel_id,Sample,Row,Scan,869.7241,909.5496,269.2486,736.6464,535.4734,559.4732,...,type,predicted0n1,p(0),p(1),CM,TPR,FNR,FDR,FPR,TNR
0,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,10,69,0.0,0.325119,-0.160987,0.347907,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
1,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,11,70,0.0,0.167710,-0.160987,0.403537,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
2,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_6...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,69,0.0,0.254875,-0.160987,0.418331,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
3,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,77,0.0,0.194011,-0.160987,0.499047,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
4,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_7...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,12,78,0.0,0.251774,0.036914,0.467859,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10297,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,138,0.0,0.439475,-0.160987,0.309653,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
10298,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,139,0.0,0.371130,0.037565,0.287210,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
10299,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,140,0.0,0.446930,0.072882,0.332403,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619
10300,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_1...,HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3,102,141,0.0,0.471105,-0.160987,0.341090,0.0,0.0,...,1,1,0.0,1.0,TP,0.406174,0.593826,0.502985,0.376381,0.623619


# 2. Prediction (made)

## deploy models for training set ##
model_xgb = jl.load(r"H:\3_output_raMSIn\modelXGBoost.joblib")
#
## deploy model ##
predicted0n1_train = model_xgb.predict(X_train)
trainDEFSDf["predicted0n1"] = predicted0n1_train
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1.csv"
trainDEFSDf.to_csv(savePath)

pTP_train = model_xgb.predict_proba(X_train)
trainDEFSDf["p(0)"] = list(pd.DataFrame(pTP_train)[0])
trainDEFSDf["p(1)"] = list(pd.DataFrame(pTP_train)[1])
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\df_train24_std_0n1_pTP.csv"
trainDEFSDf.to_csv(savePath)

In [34]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(45.38954468802699, '%')

In [35]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(28.527967918135932, '%')

In [36]:
( list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1_ROI["p(1)"]).count(1)) ) *100, "%"

(38.74246000524521, '%')

In [37]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice03_Slide3_ROI["p(1)"]).count(1)) ) *100, "%"

(97.03940982333528, '%')

In [38]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice08_Slide1_ROI["p(1)"]).count(1)) ) *100, "%"

(42.39064856711916, '%')

In [39]:
( list(HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI["p(1)"]).count(1) / (list(HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI["p(0)"]).count(1) + list(HKULiver_Post29wk_HCCLiver_FNA_Mice12_Slide5_ROI["p(1)"]).count(1)) ) *100, "%"

(5.499099936124499, '%')

# 3. EDA

In [27]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [28]:
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.cm
import numpy as np
from matplotlib.colors import ListedColormap
cmap = cm.binary

def rgb_white2alpha(rgb, ensure_increasing=False):

    # The most transparent alpha we can use is given by the min of RGB
    # Convert it from saturation to opacity
    alpha = 1. - np.min(rgb, axis=1)
    if ensure_increasing:
        # Let's also ensure the alpha value is monotonically increasing
        a_max = alpha[0]
        for i, a in enumerate(alpha):
            alpha[i] = a_max = np.maximum(a, a_max)
    alpha = np.expand_dims(alpha, -1)
    # Rescale colors to discount the white that will show through from transparency
    rgb = (rgb + alpha - 1) / alpha
    # Concatenate our alpha channel
    return np.concatenate((rgb, alpha), axis=1)
    

def cmap_white2alpha(name, ensure_increasing=False, register=True):

    # Fetch the cmap callable
    cmap = plt.get_cmap(name)
    # Get the colors out from the colormap LUT
    rgb = cmap(np.arange(cmap.N))[:, :3]  # N-by-3
    # Convert white to alpha
    rgba = rgb_white2alpha(rgb, ensure_increasing=ensure_increasing)
    # Create a new Colormap object
    cmap_alpha = matplotlib.colors.ListedColormap(rgba, name=name + "_alpha")
    if register:
        matplotlib.cm.register_cmap(name=name + "_alpha", cmap=cmap_alpha)
    return cmap_alpha

In [40]:
# Get our new version of the colormap with white2alpha
cmap_w2a = cmap_white2alpha("Reds")

In [41]:
def genAROItif(sample_ROI, sample_, name, mz, vmax_=1):

    df_heatmap = sample_ROI[["Row", "Scan", mz]]
    df_heatmap2 = pd.DataFrame(np.array([[0, 0, 0]]), columns=['Row', 'Scan', mz]).reset_index().drop(columns = ["index"])
    df_HM = df_heatmap2.loc[df_heatmap["Row"] == 0].loc[df_heatmap2["Scan"] == 0]
    Heat_Row = sample_["Row"]
    Heat_Scan = sample_["Scan"]


    #hi = 0
    for y in range(len(set(Heat_Row))):
        for x in range(1, len(set(Heat_Scan))+1):

            df_in = df_heatmap.loc[df_heatmap["Row"] == (y+1)].loc[df_heatmap["Scan"] == x]
            if len(df_in) == 1:
                df_HM = pd.concat([df_HM, df_in], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


            elif len(df_in) == 0:
                df_heatmap2["Row"] = y+1
                df_heatmap2["Scan"] = x
                df_HM = pd.concat([df_HM, df_heatmap2], join="inner", ignore_index=True)
                df_HM = df_HM.sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns = ["index"])
                #hi += 1


    #print(name, mz, hi, len(df_HM))
    if "index" in df_HM.columns:
        df_HM = df_HM.drop(columns = ["index", "Row", "Scan"])
    else:
        df_HM = df_HM.drop(columns = ["Row", "Scan"])

    df_HM = df_HM.T
    New_y = []
    for y in range(1, len(set(Heat_Row))+1):
        New_y.append(str(y))
    New_x = []
    for x in range(1, len(set(Heat_Scan))+1):
        New_x.append(str(x))

    arr = df_HM.values.copy()
    arr.resize(len(set(Heat_Row)), len(set(Heat_Scan)))

    df_HM = pd.DataFrame(arr)
    df_HM.index = New_y
    df_HM.columns = New_x

    for i in list(df_HM.columns):
        df_HM[i] = df_HM[i].astype(float)

    plt.subplots(dpi = 300)
    plt.imshow(df_HM, cmap=cmap_w2a, vmin=0, vmax=1, zorder=1) #cmap_w2a
    plt.xticks([])
    plt.yticks([])
    plt.box(False)
    plt.autoscale(enable=True, tight=True)

    plt.rcParams['figure.figsize'] = [2, 2]

    plt.savefig(f"H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_10ppm/{name}_{mz}.tif", bbox_inches = 'tight', pad_inches = 0)

In [31]:
df_raMSI_ML = pd.read_csv(r'H:/3_output_raMSIn/3_3_Output_raMSI_HKU_Ingested4FNA/0_dbMSI_10ppm/dbMSI10ppm_noLT.csv').set_index('pixel_id')
df_raMSI_ML.shape

(371480, 2603)

In [32]:
raMSI_ML = df_raMSI_ML.reset_index()

In [33]:
raMSI_ML["Sample"] = 0
raMSI_ML["Row"] = 0
raMSI_ML["Scan"] = 0
Sample = []
Row = []
Scan = []
for i in range(len(raMSI_ML["pixel_id"])):
    Sample.append("_".join(str(raMSI_ML["pixel_id"][i]).split("_")[:-2]))
    Row.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-1]))
    Scan.append(int(str(raMSI_ML["pixel_id"][i]).split("_")[-2]))
raMSI_ML["Sample"] = Sample
raMSI_ML["Row"] = Row
raMSI_ML["Scan"] = Scan

In [34]:
pop_column = raMSI_ML.pop('Scan')
raMSI_ML.insert(1, 'Scan', pop_column)
pop_column = raMSI_ML.pop('Row')
raMSI_ML.insert(1, 'Row', pop_column)
pop_column = raMSI_ML.pop('Sample')
raMSI_ML.insert(1, 'Sample', pop_column)

In [35]:
raMSI_ML = raMSI_ML.sort_values("Sample", ascending = True).reset_index().drop(columns=["index"])

In [36]:
HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4FW"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])
HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2 = raMSI_ML.loc[raMSI_ML["Sample"] == "HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2"].sort_values(["Row", "Scan"], ascending = [True, True]).reset_index().drop(columns=["index"])

In [37]:
sample_list = [HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice07_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice08_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice10_Slide5, 
               HKULiver_Post29wk_HCCAMCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide1, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide2, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_Left_MiceNL_Slide4, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice05_Slide3, 
               HKULiver_Post29wk_HCCAMCLiver_FNA_Mice09_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice01_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice08_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_Mice09_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_Mice10_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_Mice11_Slide6, 
               HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide1, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide2, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide3, 
               HKULiver_Post29wk_HCCLiver_Left_MiceHCC_Slide4, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice01_Slide3, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice06_Slide5, 
               HKULiver_Post29wk_HCCLiver_FNA_Mice11_Slide2]

In [42]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

TOP ="p(1)"
    

counter = 0

for im in range(len(sample_ROI_list)):
    start = datetime.now()

    name = name_list[im]
    sample_ = sample_list[im]
    preview = sample_ROI_list[im][[TOP]]
    for i in preview.columns:
        preview[i] = preview[i].astype(float)

    vmax_list = list(preview.describe().iloc[-1])
    vmax_ = 1
    #print(TOP, name, vmax_)

    genAROItif(sample_ROI=sample_ROI_list[im], sample_=sample_, name=name, mz=TOP, vmax_=vmax_)
    #print(df.shape)
    counter += 1
    end = datetime.now()
    Duration = end - start
    print(Duration)


0:01:09.113972
0:01:28.229262
0:02:04.584767
0:01:05.511597
0:01:18.214693
0:01:46.480771
0:00:06.502557
0:00:04.773671
0:00:07.415740
0:00:07.043556
0:03:58.605858
0:03:54.143689
0:04:10.997830
0:01:53.525182
0:01:32.220779
0:01:17.812436
0:01:16.885225
0:01:15.102415
0:01:16.742748
0:00:04.614559
0:00:11.210527
0:00:06.425741
0:00:05.430840
0:03:42.904612
0:03:49.921986
0:03:21.134278
